In [5]:
import os 
import pandas as pd 
import requests
from tqdm import tqdm

In [6]:
def get_reviews(asin,page):
    task_params = {
        'target': 'amazon_reviews',
        'query': asin,
        "domain": "com",
        "page_from": page,
        "parse": True
    }

    username = os.environ.get('username')
    password = os.environ.get('password')


    response = requests.post(
        'https://scraper-api.smartproxy.com/v2/scrape',
        json = task_params,
        auth = (username, password)
    )

    return response.json()

In [10]:
searches = os.listdir('../data/search_results')
all_reviews = {} # mapping of asin to a list of reviews to avoid duplicates 
num_pages = 1
topn = 1 # top n products for each keyword in a category 
export = []

for filename in searches:
    if '051' not in filename: continue
    products = pd.read_csv(f'../data/search_results/{filename}')
    for keyword in set(products.keyword): 
        products = products[products.keyword == keyword] 
        products = products[products.source != "paid"] # only Organic & Amazon's Choice products 
        products = products[products.reviews_count != 0] # must have at least one review 
        top = products.sort_values(by=['min_revenue', 'best_seller','rating'],ascending=[False,False,False]).head(topn)
        top = list(top['asin'])
        for asin in tqdm(top):
            if asin in all_reviews: continue
            all_reviews[asin] = [] 
            page1 = get_reviews(asin,"1")
            page1 = page1['results'][0]['content']['results']
            all_reviews[asin].extend(page1['reviews'])
            total_num_pages = int(page1['pages'])
            if total_num_pages >= 2: 
                if num_pages < total_num_pages: 
                    find_num_pages = num_pages
                else: 
                    find_num_pages = total_num_pages
                for page in range(2,find_num_pages+1): 
                    reviews = get_reviews(asin,str(page))
                    reviews = reviews['results'][0]['content']['results']['reviews']
                    all_reviews[asin].extend(reviews)

export = []
for asin, review in all_reviews.items(): 
    for r in review: 
        r['asin'] = asin
        export.append(r)
df = pd.DataFrame(export)
df.to_csv(f'../data/reviews.csv')

100%|██████████| 1/1 [00:02<00:00,  2.89s/it]
